In [389]:
VERSION

v"1.11.3"

In [390]:
] activate .

  Activating project at `~/ADMS/ADMS`


In [391]:
# ] add SymbolicUtils Latexify LaTeXStrings

In [392]:
using SymbolicUtils
using SymbolicUtils: Term, Add, Mul, Pow, Div
using Latexify
using Latexify: _latexraw
using LaTeXStrings

# Creating Symbolic variables and functions

In [393]:
function StringtoLatex(x; kwargs...)
    # A simple fallback: convert the symbolic object to a string,
    # then escape it for LaTeX.
    str = string(x)
    if length(kwargs) == 1
        for rep in kwargs[1]
            str = replace(str, rep)
        end
    end
    return str
end

StringtoLatex (generic function with 2 methods)

In [394]:
function DisplayLatex(x::SymbolicUtils.BasicSymbolic; kwargs...)
    # A simple fallback: convert the symbolic object to a string,
    # then escape it for LaTeX.
    return latexify(string(x))
end

DisplayLatex (generic function with 1 method)

In [395]:
function parse_derivative_string(s::String)
    m = match(r"^D\(\s*(.+)\s*,\s*(.+)\s*,\s*(\d+)\s*\)$", s)
    if m === nothing
        error("Input string does not match the expected pattern: D(expr, var, order)")
    end
    expr_str = m.captures[1]
    var_str = m.captures[2]
    order_str = m.captures[3]
    return expr_str, var_str, order_str
end

parse_derivative_string (generic function with 1 method)

In [396]:
function PartialDstr(der)
    println(der)
    args, dvar, order = parse_derivative_string(der)
    dorder = parse(Float32,order)
    if dorder == 0
        return der
    elseif dorder == 1.0
        return string( "\\frac{\\partial}{\\partial ", dvar,"} ", "\\left(", args,"\\right)")
    elseif isinteger(dorder)
        return string( "\\frac{\\partial^{", string(Int(dorder)), "}}{\\partial ", dvar,"} ", "\\left(", args,"\\right)")
    else
        return string( "\\frac{\\partial^{", order, "}}{\\partial ", dvar,"} ", "\\left(", args,"\\right)")
    end

end

PartialDstr (generic function with 1 method)

# Differentiation

Below is the function `Diff` and `differentiate` for symbolic differentiation with helper functions. 

The unevaluated derivative is expressed with the symbolic function `D(..,..,..)` where the 1st argument is the expression to differentiate, the second is the variable of differentiation, and the third is the order. 

In [398]:
# Declare our unevaluated derivative operator D as a symbolic function.
@syms D(.., .., ..)

#####################
# Helper Functions: #
#####################

function sym_equiv(a, b)
    # Check direct equality first.
    if Set([a]) == Set([b])
        return true
    end
    # If both are terms (i.e. composite expressions), compare their heads and their arguments.
    if SymbolicUtils.isterm(a) && SymbolicUtils.isterm(b)
        if hasproperty(a, :f) && hasproperty(b, :f) && Set([a.f]) == Set([b.f])
            if length(a.arguments) == length(b.arguments)
                return all(i -> sym_equiv(a.arguments[i], b.arguments[i]), 1:length(a.arguments))
            end
        end
    end
    return false
end

function sym_occurs_in(expr, a)
    if Set([expr]) == Set([a])
        return true
    end
    if expr isa Number
        return false
    end
    if hasproperty(expr, :arguments)
        if hasproperty(expr, :f) && expr.f == a
            return true
        end
        return any(arg -> sym_occurs_in(arg, a), expr.arguments)
    end
    return false
end

function target_occurs_in(a, var)
    if sym_equiv(a, var)
        return true
    end
    if SymbolicUtils.isterm(var) && (SymbolicUtils.symtype(var.f) <: SymbolicUtils.FnType)
        return SymbolicUtils.isterm(a) && hasproperty(a, :f) && (Set([a.f]) == Set([var.f]))
    else
        return sym_occurs_in(a, var)
    end
end

function is_symbolic_entity(x)
    if x isa SymbolicUtils.BasicSymbolic
        return true
    end
    if SymbolicUtils.isterm(x) && (SymbolicUtils.symtype(x) <: SymbolicUtils.FnType)
        return true
    end
    return false
end

function contains_symbolic_entity(x)
    if is_symbolic_entity(x)
        return true
    elseif x isa Number
        return false
    elseif x isa AbstractArray || x isa Tuple
        return any(contains_symbolic_entity, x)
    else
        return false
    end
end

function symbolic_promote(x, var)
    return x isa Number ? x * one(var) : x
end

# Helper: Check if expr is a D-term (i.e. a call to D).
function is_d_term(expr)
    return SymbolicUtils.isterm(expr) && (expr.f == D) #(Set([expr.f]) == Set([D]))
end

#############################
# differentiate (Core)      #
#############################

"""
    differentiate(expr, var)

Differentiates the symbolic expression `expr` with respect to `var`.
Handles arithmetic operations, common functions, and unknown symbolic functions.
Also, if `expr` is an unevaluated derivative (i.e. a D-term), it increments the derivative order.
"""
function differentiate(expr, var)
    if !(is_symbolic_entity(expr) || expr isa Number)
    error("First input argument $(expr) is not symbolic.")
    end
    if !(is_symbolic_entity(var) || var isa Number)
        error("Second input argument $(var) is not symbolic.")
    end

    # If expr is a number or var does not occur in expr, derivative is 0.
    if expr isa Number || !sym_occurs_in(expr, var)
        return symbolic_promote(0, var)
    # If expr is (symbolically) equivalent to var, return 1.
    elseif sym_equiv(expr, var)
        return one(expr)
    end

    # NEW: If expr is a D-term, update its derivative order.
    # if is_d_term(expr)
    #     # expr is of the form D(u, v, n)
    #     u = expr.arguments[1]
    #     v = expr.arguments[2]
    #     n = expr.arguments[3]
    #     if sym_equiv(v, var)
    #         # Increase derivative order.
    #         return D(u, v, n + 1)
    #     else
    #         # Otherwise, apply the chain rule.
    #         return D(expr, u, 1) * differentiate(u, var)
    #     end
    # end
    # NEW: D-term branch (for our unevaluated derivative operator D(expr, var, n))
    if is_d_term(expr)
        # expr is of the form D(u, v, n)
        # println("has D:")
        # println(expr)
        u = expr.arguments[1]
        v = expr.arguments[2]
        n = expr.arguments[3]
        if sym_equiv(v, var)
             return D(u, v, n + 1)
        elseif sym_occurs_in(v, var)
             return D(u, v, n + 1) * differentiate(v, var)
        else
             return symbolic_promote(0, var)
        end
    end


    # Addition
    if SymbolicUtils.isadd(expr)
        new_args = map(arg -> differentiate(arg, var), expr.arguments)
        new_args = map(x -> symbolic_promote(x, var), new_args)
        return reduce(+, new_args)
    # Multiplication (Product Rule)
    elseif SymbolicUtils.ismul(expr)
        args = expr.arguments
        terms = []
        for (i, arg) in enumerate(args)
            d_arg = differentiate(arg, var)
            prod = one(arg)
            for (j, other) in enumerate(args)
                if i != j
                    prod *= other
                end
            end
            push!(terms, d_arg * prod)
        end
        terms = map(t -> symbolic_promote(t, var), terms)
        return reduce(+, terms)
    # Exponentiation
    elseif SymbolicUtils.ispow(expr)
        base, exponent = expr.arguments
        if exponent isa Number
            return symbolic_promote(exponent, var) *
                   differentiate(base, var) *
                   SymbolicUtils.pow(base, exponent - 1)
        else
            return expr * (differentiate(exponent, var) * SymbolicUtils.log(base) +
                           exponent * differentiate(base, var) / base)
        end
    # Division
    elseif SymbolicUtils.isdiv(expr)
        num, den = expr.arguments
        return (differentiate(num, var) * den - num * differentiate(den, var)) /
               SymbolicUtils.pow(den, 2)
    # Known common functions: sin, cos, tan, exp, log.
    elseif SymbolicUtils.isterm(expr) &&
           length(expr.arguments) == 1 &&
           (expr.f in (sin, cos, tan, exp, log))
        u = expr.arguments[1]
        du = differentiate(u, var)
        if expr.f == sin
            return SymbolicUtils.cos(u) * du
        elseif expr.f == cos
            return -SymbolicUtils.sin(u) * du
        elseif expr.f == tan
            return du / SymbolicUtils.pow(SymbolicUtils.cos(u), 2)
        elseif expr.f == exp
            return SymbolicUtils.exp(u) * du
        elseif expr.f == log
            return du / u
        else
            error("Unhandled function: $(expr.f)")
        end
    # Square root: sqrt(u)
    elseif SymbolicUtils.isterm(expr) &&
           length(expr.arguments) == 1 &&
           expr.f == sqrt
        u = expr.arguments[1]
        du = differentiate(u, var)
        return du / (2 * SymbolicUtils.sqrt(u))
    # Inverse trigonometric/hyperbolic functions.
    elseif SymbolicUtils.isterm(expr) &&
           length(expr.arguments) == 1 &&
           (expr.f in (asin, acos, atan, sinh, cosh, tanh, asinh, acosh, atanh))
        u = expr.arguments[1]
        du = differentiate(u, var)
        if expr.f == asin
            return du / SymbolicUtils.sqrt(1 - SymbolicUtils.pow(u, 2))
        elseif expr.f == acos
            return -du / SymbolicUtils.sqrt(1 - SymbolicUtils.pow(u, 2))
        elseif expr.f == atan
            return du / (1 + SymbolicUtils.pow(u, 2))
        elseif expr.f == sinh
            return SymbolicUtils.cosh(u) * du
        elseif expr.f == cosh
            return SymbolicUtils.sinh(u) * du
        elseif expr.f == tanh
            return du / SymbolicUtils.pow(SymbolicUtils.cosh(u), 2)
        elseif expr.f == asinh
            return du / SymbolicUtils.sqrt(1 + SymbolicUtils.pow(u, 2))
        elseif expr.f == acosh
            return du / SymbolicUtils.sqrt(SymbolicUtils.pow(u, 2) - 1)
        elseif expr.f == atanh
            return du / (1 - SymbolicUtils.pow(u, 2))
        else
            error("Unhandled function: $(expr.f)")
        end
    # Unknown symbolic functions (created via @syms)
    elseif SymbolicUtils.isterm(expr) && length(expr.arguments) >= 1 &&
           (SymbolicUtils.symtype(expr.f) <: SymbolicUtils.FnType)
        idx = findfirst(a -> target_occurs_in(a, var), expr.arguments)
        if idx === nothing
            return symbolic_promote(0, var)
        else
            u = expr.arguments[idx]
            return D(expr, u, 1) * differentiate(u, var)
        end
    # Otherwise, atomic variable: return 1 if equal to var, else 0.
    else
        return Set([expr]) == Set([var]) ? one(expr) : symbolic_promote(0, var)
    end
end

##############################
# Diff (High-Level Function) #
##############################


function process_dv(e, dv)
    if dv isa AbstractVector
        # println("process_dv: dv")
        # println(dv)
        # println("length(dv): ",length(dv))
        # println("dv[2] isa Integer: ",dv[2] isa Integer)
        if length(dv) == 2 && dv[2] isa Integer
            dvar = dv[1]
            n = dv[2]
            res = e
            # println("res: ",res)
            for i in 1:n
                res = differentiate(res, dvar)
                # println("d$(n)/d$(dvar) = ",res)
                res = simplify(res)
            end
            return res
        else
            error("Invalid vector format. Expected [dvar, n] with n an integer.")
        end
    else
        return differentiate(e, dv)
    end
end

function Diff(expr, dvars...)
    if !contains_symbolic_entity(expr)
        error("Expected first argument expr to contain a SymbolicUtils entity.")
    end
    if !contains_symbolic_entity(dvars)
        error("Expected differentiation variables to contain a SymbolicUtils entity.")
    end
    if length(dvars) == 0
        return expr
    end
    res = expr
    if length(dvars) == 1
        res = process_dv(res, dvars[1])
    else
        for dv in dvars
            res = process_dv(res, dv)
        end
    end
    return res
end


Diff (generic function with 1 method)

## Basic Functions

In [399]:
basicfuncs = [x, x^2, x*y, x/y, sqrt(x),exp(x), log(x), cos(x), sin(x), tan(x), acos(x), asin(x), atan(x), cosh(x), sinh(x), tanh(x), acosh(x), asinh(x), atanh(x)]

19-element Vector{SymbolicUtils.BasicSymbolic{Number}}:
 x
 x^2
 x*y
 x / y
 sqrt(x)
 exp(x)
 log(x)
 cos(x)
 sin(x)
 tan(x)
 acos(x)
 asin(x)
 atan(x)
 cosh(x)
 sinh(x)
 tanh(x)
 acosh(x)
 asinh(x)
 atanh(x)

In [400]:
print(string(x^2))

x^2

In [401]:
latexstring(string(x^2))

L"$x^2$"

In [402]:
stringsub = [r"exp\((.*?)\)" => x -> "e^{" * x[5:end-1] * "} ", 
    r"exp\((.*?)\)" => x -> "e^{" * x[5:end-1] * "} ", 
    r"^D\(\s*(.+)\s*,\s*(.+)\s*,\s*(\d+)\s*\)$" => x -> PartialDstr(string(x)),
    r"sqrt\((.*?)\)" => x -> "\\sqrt{" * x[6:end-1] * "} ",
    r"cos\((.*?)\)" => x -> "\\cos{\\left(" * x[5:end-1] * "\\right)} ",
    r"sin\((.*?)\)" => x -> "\\sin{\\left(" * x[5:end-1] * "\\right)} ",
    r"tan\((.*?)\)" => x -> "\\tan{\\left(" * x[5:end-1] * "\\right)} ",
    r"acos\((.*?)\)" => x -> "\\acos{\\left(" * x[6:end-1] * "\\right)} ",
    r"asin\((.*?)\)" => x -> "\\asin{\\left(" * x[6:end-1] * "\\right)} ",
    r"atan\((.*?)\)" => x -> "\\atan{\\left(" * x[6:end-1] * "\\right)} ",
    r"cosh\((.*?)\)" => x -> "\\cosh{\\left(" * x[6:end-1] * "\\right)} ",
    r"sinh\((.*?)\)" => x -> "\\sinh{\\left(" * x[6:end-1] * "\\right)} ",
    r"tanh\((.*?)\)" => x -> "\\tanh{\\left(" * x[6:end-1] * "\\right)} "
    ]

13-element Vector{Pair{Regex, Function}}:
                             r"exp\((.*?)\)" => var"#242#255"()
                             r"exp\((.*?)\)" => var"#243#256"()
 r"^D\(\s*(.+)\s*,\s*(.+)\s*,\s*(\d+)\s*\)$" => var"#244#257"()
                            r"sqrt\((.*?)\)" => var"#245#258"()
                             r"cos\((.*?)\)" => var"#246#259"()
                             r"sin\((.*?)\)" => var"#247#260"()
                             r"tan\((.*?)\)" => var"#248#261"()
                            r"acos\((.*?)\)" => var"#249#262"()
                            r"asin\((.*?)\)" => var"#250#263"()
                            r"atan\((.*?)\)" => var"#251#264"()
                            r"cosh\((.*?)\)" => var"#252#265"()
                            r"sinh\((.*?)\)" => var"#253#266"()
                            r"tanh\((.*?)\)" => var"#254#267"()

## Playing around with Latex Display

These Latex functions seem to work for the most part but fail in some cases

In [403]:
for ff in basicfuncs
    lstr = StringtoLatex(Diff(ff,x); stringsub);
    dlstr = StringtoLatex((D(ff,x,1));stringsub);
    flstr = string(dlstr," = ",lstr);
    # latexify(lstr)
    display(latexstring(flstr));

end

D(x, x, 1)


L"$\frac{\partial}{\partial x} \left(x\right) = 1$"

D(x^2, x, 1)


L"$\frac{\partial}{\partial x} \left(x^2\right) = 2x$"

D(x*y, x, 1)


...

L"$\frac{\partial}{\partial x} \left(a\cosh{\left(x\right)} \right) = 1 / \sqrt{-1 + x^2} $"

D(asinh(x), x, 1)


L"$\frac{\partial}{\partial x} \left(a\sinh{\left(x\right)} \right) = 1 / \sqrt{1 + x^2} $"

D(atanh(x), x, 1)


L"$\frac{\partial}{\partial x} \left(a\tanh{\left(x\right)} \right) = 1 / (1 - (x^2))$"

## Symbolic Functions

In [404]:
fx = f(x)
gxy = g(x,y)
hxyz = h(x,y,z)
fgxy = f(gxy)
expr = fgxy + z^2 + fx

functests = [fx, gxy, hxyz, fgxy, expr]

5-element Vector{SymbolicUtils.BasicSymbolic{Number}}:
 f(x)
 g(x, y)
 h(x, y, z)
 f(g(x, y))
 f(x) + f(g(x, y)) + z^2

In [405]:
for ff in functests

    println("d/dy ",ff, " = ",Diff(ff,y))

end

d/dy f(x) = 0
(x, y) = D(g(x, y), y, 1)
d/dy h(x, y, z) = D(h(x, y, z), y, 1)
d/dy f(g(x, y)) = D(f(g(x, y)), g(x, y), 1)*D(g(x, y), y, 1)
f(x) + f(g(x, y)) + z^2 = D(f(g(x, y)), g(x, y), 1)*D(g(x, y), y, 1)


In [406]:
for ff in functests

    println("d/dz ",ff, " = ",Diff(ff,z))

end

d/dz f(x) = 0
d/dz g(x, y) = 0
d/dz h(x, y, z) = D(h(x, y, z), z, 1)
d/dz f(g(x, y)) = 0
d/dz f(x) + f(g(x, y)) + z^2 = 2z


In [407]:
Diff(fx,[x,2])

D(f(x), x, 2)

## nth order derivatives

In [408]:
expr0 = x^2
expr1 = x^3
expr2 = fgxy + z^2 + fx
expr3 = fx + hxyz^2 + cos(y)
expr4 = log(x*fx)
expr5 = exp(-x*fx)
expr6 = exp(x*z + gxy)

exprtests = [expr0, expr1, expr2, expr3, expr4, expr5, expr6]

7-element Vector{SymbolicUtils.BasicSymbolic{Number}}:
 x^2
 x^3
 f(x) + f(g(x, y)) + z^2
 cos(y) + f(x) + h(x, y, z)^2
 log(x*f(x))
 exp(-x*f(x))
 exp(g(x, y) + x*z)

In [409]:
for ff in exprtests
    # println(ff)
    println("d2/dx2 ",ff, " = ",Diff(ff,[x,2]))

end

d2/dx2 x^2 = 2
d2/dx2 x^3 = 6x
f(x) + f(g(x, y)) + z^2 = D(f(x), x, 2) + D(f(g(x, y)), g(x, y), 1)*D(g(x, y), x, 2) + (D(g(x, y), x, 1)^2)*D(f(g(x, y)), g(x, y), 2)
d2/dx2 cos(y) + f(x) + h(x, y, z)^2 = D(f(x), x, 2) + 2(D(h(x, y, z), x, 1)^2) + 2D(h(x, y, z), x, 2)*h(x, y, z)
d2/dx2 log(x*f(x)) = (-(f(x)^2) - (x^2)*(D(f(x), x, 1)^2) + (x^2)*D(f(x), x, 2)*f(x)) / ((x^2)*(f(x)^2))
d2/dx2 exp(-x*f(x)) = (-2D(f(x), x, 1) - x*D(f(x), x, 2))*exp(-x*f(x)) + ((-f(x) - x*D(f(x), x, 1))^2)*exp(-x*f(x))
exp(g(x, y) + x*z) = (D(g(x, y), x, 2) + (z + D(g(x, y), x, 1))^2)*exp(g(x, y) + x*z)


In [410]:
for ff in exprtests
    # println(ff)
    println("d2/dxdy ",ff, " = ",Diff(ff,[x,1],[y,1]))

end

d2/dxdy x^2 = 0
d2/dxdy x^3 = 0
d2/dxdy f(x) + f(g(x, y)) + z^2 = D(g(x, y), x, 1)*D(f(g(x, y)), g(x, y), 2)*D(g(x, y), y, 1)
d2/dxdy cos(y) + f(x) + h(x, y, z)^2 = 2D(h(x, y, z), y, 1)*D(h(x, y, z), x, 1)
d2/dxdy log(x*f(x)) = 0
d2/dxdy exp(-x*f(x)) = 0
(g(x, y) + x*z) = (z + D(g(x, y), x, 1))*exp(g(x, y) + x*z)*D(g(x, y), y, 1)


# Integration

Below `Integ` and `integrate` are functions to do symbolic integration. 

The unevaluated integral is `It(..,..)` where the first argument is the expresion to be integrated and the second is the variable of integration. As opposed to `D(..,..,..)`, integrals will have no "order" and instead will be nested. 

In [411]:
# Declare the unevaluated integral operator It.
@syms It(.., ..)

#############################
# Integration (Core)        #
#############################

"""
    integrate(expr, dvar)

Computes the antiderivative (indefinite integral) of the symbolic expression `expr`
with respect to the variable `dvar`. When a known antiderivative rule applies,
the result is fully evaluated; otherwise, the result is returned unevaluated
as It(expr, dvar).
"""
function integrate(expr, dvar)
    # Check that inputs are symbolic.
    if !is_symbolic_entity(expr)
        error("First argument $(expr) is not symbolic.")
    elseif !is_symbolic_entity(dvar)
        error("Second argument $(dvar) is not symbolic.")
    end

    # If expr does not depend on dvar, then ∫ expr dx = expr * dvar.
    if expr isa Number || !sym_occurs_in(expr, dvar)
        return expr * dvar
    end

    # If expr is exactly dvar, then ∫ dvar dx = dvar^2/2.
    if sym_equiv(expr, dvar)
        return SymbolicUtils.pow(dvar, 2) / 2
    end

    # Linearity: ∫ (f + g) dx = ∫ f dx + ∫ g dx.
    if SymbolicUtils.isadd(expr)
        new_args = map(arg -> integrate(arg, dvar), expr.arguments)
        return reduce(+, new_args)
    end

    # Constant factor extraction for products:
    if SymbolicUtils.ismul(expr)
        const_factors = []
        var_factors = []
        for f in expr.arguments
            if !sym_occurs_in(f, dvar)
                push!(const_factors, f)
            else
                push!(var_factors, f)
            end
        end
        if !isempty(const_factors) && length(var_factors) == 1
            constant_part = reduce(*, const_factors)
            return constant_part * integrate(var_factors[1], dvar)
        end
        return It(expr, dvar)
    end

    # Power rule: if expr = dvar^n and n ≠ -1.
    if SymbolicUtils.ispow(expr)
        base, exponent = expr.arguments
        if sym_equiv(base, dvar) && exponent isa Number && exponent != -1
            return SymbolicUtils.pow(dvar, exponent + 1) / (exponent + 1)
        else
            return It(expr, dvar)
        end
    end

    # Known common functions (when the argument is exactly dvar):
    if SymbolicUtils.isterm(expr) && length(expr.arguments) == 1
        u = expr.arguments[1]
        if sym_equiv(u, dvar)
            if expr.f == sin
                # ∫ sin(x) dx = -cos(x)
                return -SymbolicUtils.cos(dvar)
            elseif expr.f == cos
                # ∫ cos(x) dx = sin(x)
                return SymbolicUtils.sin(dvar)
            elseif expr.f == tan
                # ∫ tan(x) dx = -log(|cos(x)|) (ignoring absolute value)
                return -SymbolicUtils.log(SymbolicUtils.cos(dvar))
            elseif expr.f == asin
                # ∫ asin(x) dx = x*asin(x) + sqrt(1-x^2)
                return dvar * expr + SymbolicUtils.sqrt(1 - SymbolicUtils.pow(dvar, 2))
            elseif expr.f == acos
                # ∫ acos(x) dx = x*acos(x) - sqrt(1-x^2)
                return dvar * expr - SymbolicUtils.sqrt(1 - SymbolicUtils.pow(dvar, 2))
            elseif expr.f == atan
                # ∫ atan(x) dx = x*atan(x) - 1/2 log(1+x^2)
                return dvar * expr - (1/2) * SymbolicUtils.log(1 + SymbolicUtils.pow(dvar, 2))
            elseif expr.f == sinh
                # ∫ sinh(x) dx = cosh(x)
                return SymbolicUtils.cosh(dvar)
            elseif expr.f == cosh
                # ∫ cosh(x) dx = sinh(x)
                return SymbolicUtils.sinh(dvar)
            elseif expr.f == tanh
                # ∫ tanh(x) dx = log(cosh(x))
                return SymbolicUtils.log(SymbolicUtils.cosh(dvar))
            elseif expr.f == asinh
                # ∫ asinh(x) dx = x*asinh(x) - sqrt(1+x^2)
                return dvar * expr - SymbolicUtils.sqrt(1 + SymbolicUtils.pow(dvar, 2))
            elseif expr.f == acosh
                # ∫ acosh(x) dx = x*acosh(x) - sqrt(x^2-1)
                return dvar * expr - SymbolicUtils.sqrt(SymbolicUtils.pow(dvar, 2) - 1)
            elseif expr.f == atanh
                # ∫ atanh(x) dx = x*atanh(x) + 1/2 log(1-x^2)
                return dvar * expr + (1/2) * SymbolicUtils.log(1 - SymbolicUtils.pow(dvar, 2))
            else
                return It(expr, dvar)
            end
        else
            return It(expr, dvar)
        end
    end

    # For any other case (including unknown functions), return the unevaluated integral.
    return It(expr, dvar)
end

##############################
# Int (High-Level Function)  #
##############################

# For integration instructions, we expect each to be either a symbolic variable/expression or
# a single-element vector [dvar].
function process_iv(e, dv)
    if dv isa AbstractVector
        if length(dv) == 1
            dvar = dv[1]
            return integrate(e, dvar)
        else
            error("Invalid vector format for integration. Expected [dvar].")
        end
    else
        return integrate(e, dv)
    end
end

"""
    Int(expr, dvars...)

Computes the iterated integral of the symbolic expression `expr` with respect to
one or more variables. If a known integration rule applies, the antiderivative is fully
evaluated; otherwise, the result is left unevaluated as It(expr, dvar).
"""
function Integ(expr, dvars...)
    if !contains_symbolic_entity(expr)
        error("Expected first argument expr to contain a SymbolicUtils entity.")
    end
    if length(dvars) == 0
        return expr
    end
    res = expr
    if length(dvars) == 1
        res = process_iv(res, dvars[1])
    else
        for dv in dvars
            res = process_iv(res, dv)
        end
    end
    return res
end


Integ

## Basic Functions

In [412]:
for ff in basicfuncs

    println("Int( ",ff, " dx) = ",Integ(ff,x))

end

Int( x dx) = (1//2)*(x^2)
Int( x^2 dx) = (1//3)*(x^3)
Int( x*y dx) = (1//2)*(x^2)*y
Int( x / y dx) = It(x / y, x)
Int( sqrt(x) dx) = It(sqrt(x), x)
Int( exp(x) dx) = It(exp(x), x)
Int( log(x) dx) = It(log(x), x)
Int( cos(x) dx) = sin(x)
Int( sin(x) dx) = -cos(x)
Int( tan(x) dx) = -log(cos(x))
Int( acos(x) dx) = -sqrt(1 - (x^2)) + x*acos(x)
Int( asin(x) dx) = sqrt(1 - (x^2)) + x*asin(x)
Int( atan(x) dx) = -0.5log(1 + x^2) + x*atan(x)
Int( cosh(x) dx) = sinh(x)
Int( sinh(x) dx) = cosh(x)
Int( tanh(x) dx) = log(cosh(x))
Int( acosh(x) dx) = -sqrt(-1 + x^2) + x*acosh(x)
Int( asinh(x) dx) = -sqrt(1 + x^2) + x*asinh(x)
Int( atanh(x) dx) = 0.5log(1 - (x^2)) + x*atanh(x)


## Symbolic Functions

In [424]:
for ff in exprtests

    println("Int( ",ff, " dx ) = ",Integ(ff,x))

end

Int( x^2 dx ) = (1//3)*(x^3)
Int( x^3 dx ) = (1//4)*(x^4)
Int( f(x) + f(g(x, y)) + z^2 dx ) = It(f(g(x, y)), x) + It(f(x), x) + x*(z^2)
Int( cos(y) + f(x) + h(x, y, z)^2 dx ) = It(h(x, y, z)^2, x) + It(f(x), x) + x*cos(y)
Int( log(x*f(x)) dx ) = It(log(x*f(x)), x)
Int( exp(-x*f(x)) dx ) = It(exp(-x*f(x)), x)
Int( exp(g(x, y) + x*z) dx ) = It(exp(g(x, y) + x*z), x)
